In [1]:
import numpy as np, pandas as pd

In [2]:
dataset_name = "ionosphere"

data_filename = "../data/{}0.csv".format(dataset_name)

data_raw = pd.read_csv(data_filename)

class_prepared = pd.get_dummies(data_raw.iloc[:,-1])
data_mod = data_raw.iloc[:,:-1]

data_mod.to_csv("../data/{}_train.tab".format(dataset_name), sep=" ", index=False)
class_prepared.to_csv("../data/{}_train.Y".format(dataset_name), sep=" ", index=False)

In [3]:
%run ../apyori.py

# below function basically takes a data frame and a support threshold and returns itemsets which satisfy the threshold
def run_apriori(df, support_thres):
    # the idea is to basically make a list of strings out of df and run apriori api on it 
    # return the frequent itemsets
    dataset = []
    for i in range(0,df.shape[0]):
        temp = []
        for col_name in df.columns:
            temp.append(col_name+"="+str(df[col_name][i]))
        dataset.append(temp)

    results = list(apriori(dataset, min_support=support_thres))
    
    list_itemsets = []
    for ele in results:
        temp = []
        for pred in ele.items:
            temp.append(pred)
        list_itemsets.append(temp)

    return list_itemsets

In [4]:
import fim

def process_dataset(df):
    dataset = []
    for i in range(0,df.shape[0]):
        temp = []
        for col_name in df.columns:
            temp.append(col_name+"="+str(df[col_name][i]))
        dataset.append(temp)
        
    return dataset
    

def run_fim_apriori(df, support_thres):
    # the idea is to basically make a list of strings out of df and run apriori api on it 
    # return the frequent itemsets
    dataset = []
    for i in range(0,df.shape[0]):
        temp = []
        for col_name in df.columns:
            temp.append(col_name+"="+str(df[col_name][i]))
        dataset.append(temp)

    result_raw = fim.apriori(dataset, supp=support_thres)
    result = list(map(lambda i: list(i[0]), result_raw))
        
    return result

In [5]:
df = pd.read_csv('../titanic_train.tab',' ', header=None, names=['Passenger_Cat', 'Age_Cat', 'Gender'])
df1 = pd.read_csv('../titanic_train.Y', ' ', header=None, names=['Died', 'Survived'])
Y = list(df1['Died'].values)

itemsets = run_apriori(df, 0.1)

itemsets

[['Age_Cat=adult'],
 ['Gender=female'],
 ['Gender=male'],
 ['Passenger_Cat=1st_class'],
 ['Passenger_Cat=2nd_class'],
 ['Passenger_Cat=3rd_class'],
 ['Passenger_Cat=crew'],
 ['Gender=female', 'Age_Cat=adult'],
 ['Age_Cat=adult', 'Gender=male'],
 ['Passenger_Cat=1st_class', 'Age_Cat=adult'],
 ['Age_Cat=adult', 'Passenger_Cat=2nd_class'],
 ['Age_Cat=adult', 'Passenger_Cat=3rd_class'],
 ['Age_Cat=adult', 'Passenger_Cat=crew'],
 ['Passenger_Cat=3rd_class', 'Gender=male'],
 ['Passenger_Cat=crew', 'Gender=male'],
 ['Age_Cat=adult', 'Passenger_Cat=3rd_class', 'Gender=male'],
 ['Age_Cat=adult', 'Passenger_Cat=crew', 'Gender=male']]

In [6]:
df = pd.read_csv('../data/iris0.csv')

df_raw = df.iloc[:, :-1]
Y = df.iloc[:, -1]

def run_fim_apriori(df, minsup):
    processed_df = process_dataset(df)
    
    result_raw = fim.apriori(processed_df, supp=(minsup*100))
    result = list(map(lambda i: list(i[0]), result_raw))
    
    return result


rules = run_fim_apriori(df_raw, 0.2)

len(rules)



27

In [7]:
%run ../IDS_smooth_local.py

list_of_rules = createrules(rules, list(set(Y)))

len(list_of_rules)

81

In [8]:


#!more "../train_model_SLS.py"



print("----------list of rules------------")

print("-------------rules created---------")
for r in list_of_rules:
    r.print_rule()

lambda_array = [1.0]*7     # use separate hyperparamter search routine
s1 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, 0.33)
s2 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, -1.0)
f1 = func_evaluation(s1, list_of_rules, df, Y, lambda_array)
f2 = func_evaluation(s2, list_of_rules, df, Y, lambda_array)

result_set = {}
if f1 > f2:
    print("The Solution Set is: "+str(s1))
    result_set = list(s1)
else:
    print("The Solution Set is: "+str(s2))
    result_set = list(s2)

----------list of rules------------
-------------rules created---------
If sepalwidth == 3.35_to_inf, then Iris-setosa
If sepalwidth == 3.35_to_inf, then Iris-versicolor
If sepalwidth == 3.35_to_inf, then Iris-virginica
If sepalwidth == 3.35_to_inf and petalwidth == -inf_to_0.8 and petallength == -inf_to_2.45, then Iris-setosa
If sepalwidth == 3.35_to_inf and petalwidth == -inf_to_0.8 and petallength == -inf_to_2.45, then Iris-versicolor
If sepalwidth == 3.35_to_inf and petalwidth == -inf_to_0.8 and petallength == -inf_to_2.45, then Iris-virginica
If sepalwidth == 3.35_to_inf and petalwidth == -inf_to_0.8, then Iris-setosa
If sepalwidth == 3.35_to_inf and petalwidth == -inf_to_0.8, then Iris-versicolor
If sepalwidth == 3.35_to_inf and petalwidth == -inf_to_0.8, then Iris-virginica
If sepalwidth == 3.35_to_inf and petallength == -inf_to_2.45, then Iris-setosa
If sepalwidth == 3.35_to_inf and petallength == -inf_to_2.45, then Iris-versicolor
If sepalwidth == 3.35_to_inf and petallength =

Standard Error 276.67235767239197
Standard Error 257.49695369071844
Omega estimate is -262.18
Estimating omega for rule 9
Standard Error 665.8112923343971
Standard Error 473.91538458885253
Standard Error 389.2289672235687
Standard Error 343.2919569905112
Standard Error 306.7508426068297
Standard Error 268.9008827444431
Omega estimate is 412.6
Estimating omega for rule 10
Standard Error 700.5624868632348
Standard Error 541.2503389144435
Standard Error 430.3651531828223
Standard Error 401.40279691975246
Standard Error 355.6975817629352
Standard Error 310.44292875485
Standard Error 284.21260439260396
Standard Error 261.7932278580631
Omega estimate is -498.525
Estimating omega for rule 11
Standard Error 483.42193268406845
Standard Error 376.27816090626357
Standard Error 357.0228185466063
Standard Error 318.43023213001464
Standard Error 287.3362744799201
Standard Error 269.5390241449966
Omega estimate is 205.966666667
Estimating omega for rule 12
Standard Error 637.7244616603632
Standard Er

KeyboardInterrupt: 

In [8]:
def optimize():
    lambda_array = [1.0]*7     # use separate hyperparamter search routine
    s1 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, 0.33)
    s2 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, -1.0)
    f1 = func_evaluation(s1, list_of_rules, df, Y, lambda_array)
    f2 = func_evaluation(s2, list_of_rules, df, Y, lambda_array)

    result_set = {}
    if f1 > f2:
        print("The Solution Set is: "+str(s1))
        result_set = list(s1)
    else:
        print("The Solution Set is: "+str(s2))
        result_set = list(s2)

In [9]:
import cProfile

cProfile.run("optimize()")

2/n*n OPT value is 541.2068282274043
Estimating omega for rule 0
Standard Error 701.9606171289099
Standard Error 590.5271533765404
Standard Error 433.57431393442084
Standard Error 377.2715354812433
Standard Error 332.0338274935251
Standard Error 295.17255309708565
Standard Error 274.4707261496562
Standard Error 255.25992963974488
Omega estimate is 172.175
Estimating omega for rule 1
Standard Error 747.8983848090594
Standard Error 492.25769166159307
Standard Error 402.88993387129295
Standard Error 347.4396650634107
Standard Error 302.4987907413846
Standard Error 267.9905946905462
Omega estimate is -325.766666667
Estimating omega for rule 2
Standard Error 488.53764747458314
Standard Error 383.3110334636873
Standard Error 362.0255783883335
Standard Error 337.2059073057217
Standard Error 290.6082092164638
Standard Error 266.9114792051505
Omega estimate is -643.966666667
Estimating omega for rule 3
Standard Error 553.672863702024
Standard Error 531.5478406267869
Standard Error 413.431360687

Standard Error 454.3959736562374
Standard Error 413.8569147977667
Standard Error 341.02711319996246
Standard Error 297.25203913177785
Standard Error 272.224956386496
Standard Error 250.307334100248
Omega estimate is -449.4
Estimating omega for rule 30
Standard Error 684.1760190184979
Standard Error 500.00288974164937
Standard Error 408.3947637238228
Standard Error 396.92796809921947
Standard Error 341.64139130965964
Standard Error 307.6498447835955
Standard Error 281.78413780371716
Standard Error 253.75967123657327
Omega estimate is 144.575
Estimating omega for rule 31
Standard Error 575.2410599044543
Standard Error 434.5996163136825
Standard Error 343.2870344187208
Standard Error 339.62242246905606
Standard Error 298.6472251202077
Standard Error 270.0835401452806
Omega estimate is 91.0333333332
Estimating omega for rule 32
Standard Error 524.6234887993484
Standard Error 341.15030228625034
Standard Error 354.42824860246026
Standard Error 339.31649567089283
Standard Error 294.9745040778

Standard Error 437.83418094068446
Standard Error 432.7142781328114
Standard Error 423.8101044449384
Standard Error 347.02209763932905
Standard Error 319.7381856957345
Standard Error 283.74202100055874
Standard Error 253.65551914096125
Omega estimate is -214.628571429
Estimating omega for rule 59
Standard Error 376.4916798549471
Standard Error 462.0739695925318
Standard Error 394.0428719389883
Standard Error 319.06525659925745
Standard Error 296.5922547471528
Standard Error 265.4807140934263
Omega estimate is -397.6
Estimating omega for rule 60
Standard Error 708.4736275966806
Standard Error 468.2809410492808
Standard Error 396.76623423576456
Standard Error 339.03423580351733
Standard Error 289.77418344635186
Standard Error 268.5989038390115
Omega estimate is -462.083333333
Estimating omega for rule 61
Standard Error 752.331233566705
Standard Error 525.8713524950755
Standard Error 404.32967686356216
Standard Error 344.21252190234884
Standard Error 313.23968507199083
Standard Error 285.0

Standard Error 303.9441236280116
Standard Error 273.2705359942645
Standard Error 249.7807676761256
Omega estimate is -312.371428571
Estimating omega for rule 7
Standard Error 884.8021609376867
Standard Error 498.89422400945875
Standard Error 410.15475855213845
Standard Error 336.86066384152065
Standard Error 300.2898079389309
Standard Error 291.6267524673099
Standard Error 266.4818232386931
Omega estimate is -631.257142857
Estimating omega for rule 8
Standard Error 580.2992029979018
Standard Error 467.4850215514931
Standard Error 395.9703077626271
Standard Error 332.06630604349937
Standard Error 302.9350781537193
Standard Error 277.4398128757272
Standard Error 252.2548870041821
Omega estimate is -391.4
Estimating omega for rule 9
Standard Error 488.24719763660704
Standard Error 430.5708424580094
Standard Error 365.1365591827965
Standard Error 352.1616350307058
Standard Error 309.86004356805995
Standard Error 272.93995023665036
Standard Error 262.51691817756443
Omega estimate is 42.1285

Standard Error 649.3251427443727
Standard Error 514.8886945253313
Standard Error 398.30543658706796
Standard Error 335.3311167711699
Standard Error 286.5662852465377
Standard Error 258.0224241259892
Omega estimate is -565.833333333
Estimating omega for rule 35
Standard Error 476.7339425717451
Standard Error 403.66510609043235
Standard Error 353.6736889553126
Standard Error 304.1226108362793
Standard Error 301.50025793687144
Standard Error 280.27108430780737
Standard Error 266.6223146085682
Omega estimate is -81.5428571429
Estimating omega for rule 36
Standard Error 560.232744491073
Standard Error 488.4255979675922
Standard Error 403.4890303708759
Standard Error 342.4006424999448
Standard Error 298.87352392609154
Standard Error 289.1141177917147
Standard Error 273.0885984341935
Standard Error 266.15617373707374
Omega estimate is -288.2625
Estimating omega for rule 37
Standard Error 543.7758812231377
Standard Error 467.2507718024658
Standard Error 385.2804737869303
Standard Error 373.773

Standard Error 343.6341293348931
Standard Error 291.77329226644446
Standard Error 276.09971821453956
Standard Error 253.63475809344337
Omega estimate is -594.828571429
Estimating omega for rule 63
Standard Error 499.1762864960635
Standard Error 486.38539233307574
Standard Error 402.43329960333756
Standard Error 362.2558017996399
Standard Error 335.91329006158713
Standard Error 307.2363346266434
Standard Error 270.76995144125306
Standard Error 246.9631178419538
Omega estimate is -138.475
Estimating omega for rule 64
Standard Error 710.2936744755651
Standard Error 412.8091326509141
Standard Error 392.8596227002059
Standard Error 330.4778182325555
Standard Error 292.26188635537136
Standard Error 270.6113866647015
Standard Error 252.5714185866652
Omega estimate is -245.514285714
Estimating omega for rule 65
Standard Error 780.602109272067
Standard Error 531.2006554730519
Standard Error 426.26265268502476
Standard Error 349.57115144398716
Standard Error 308.62003995852245
Standard Error 282

      138    0.000    0.000    0.000    0.000 generic.py:414(_info_axis)
     1618    0.001    0.000    0.003    0.000 generic.py:7(_check)
       81    0.000    0.000    0.000    0.000 indexing.py:1891(convert_to_index_sliceable)
       81    0.000    0.000    0.001    0.000 indexing.py:1928(check_bool_indexer)
       81    0.002    0.000    0.004    0.000 indexing.py:1981(maybe_convert_indices)
      142    0.000    0.000    0.000    0.000 inference.py:364(is_hashable)
      223    0.001    0.000    0.002    0.000 internals.py:102(__init__)
       81    0.001    0.000    0.011    0.000 internals.py:1025(take_nd)
      357    0.000    0.000    0.001    0.000 internals.py:169(to_dense)
       81    0.000    0.000    0.000    0.000 internals.py:181(fill_value)
       85    0.000    0.000    0.001    0.000 internals.py:1838(__init__)
      466    0.000    0.000    0.000    0.000 internals.py:185(mgr_locs)
       85    0.000    0.000    0.002    0.000 internals.py:214(make_block_same_clas

In [55]:
np_rules = np.array(list_of_rules)
solution_rules = np_rules[result_set]

list(map(lambda r: r.print_rule(), solution_rules))

If petalwidth == 0.8_to_1.75, then Iris-virginica
If petalwidth == 0.8_to_1.75, then Iris-versicolor
If petallength == 4.75_to_inf, then Iris-setosa
If sepalwidth == 2.95_to_3.35, then Iris-virginica
If sepalwidth == 2.95_to_3.35, then Iris-versicolor
If sepallength == -inf_to_5.55, then Iris-virginica


[None, None, None, None, None, None]

In [60]:
r = np_rules[0]


set(r.itemset) 

{('petalwidth', '0.8_to_1.75')}

In [87]:
def prepare_dataset_to_tuples(df):
    df_dict = df.iloc[:, :-1].to_dict(orient="records")
    df_items = list(map(lambda i: set(i.items()), df_dict))

    return df_items


def predict(rule_list, instance):
    matches = []
    
    for rule in rule_list:
        if r.itemset <= instance:
            matches.append(r)
            
    return matches


df_items = prepare_dataset_to_tuples(df)


predict(solution_rules, df_items[3])

print(df_items[1])

list(map(lambda r: r.print_rule(), solution_rules))

{('petallength', '-inf_to_2.45'), ('sepalwidth', '2.95_to_3.35'), ('petalwidth', '-inf_to_0.8'), ('sepallength', '-inf_to_5.55')}
If petalwidth == 0.8_to_1.75, then Iris-virginica
If petalwidth == 0.8_to_1.75, then Iris-versicolor
If petallength == 4.75_to_inf, then Iris-setosa
If sepalwidth == 2.95_to_3.35, then Iris-virginica
If sepalwidth == 2.95_to_3.35, then Iris-versicolor
If sepallength == -inf_to_5.55, then Iris-virginica


[None, None, None, None, None, None]

In [145]:
def get_cover(r, df):
    dfnew = df.copy()
    for pattern in r.itemset: 
        dfnew = dfnew[dfnew[pattern[0]] == pattern[1]]
    return list(dfnew.index.values)

def get_cover_faster(r, df):
    mask = np.ones(len(df), dtype=bool)
    for pattern in r.itemset:
        mask &= df[pattern[0]] == pattern[1]
        
    return list(df[mask].index.values)


cache = {}
def get_cover_memoized(r, df):    
    result = cache.get(repr(r))
    
    if result:
        return result
    
    mask = np.ones(len(df), dtype=bool)
    for pattern in r.itemset:
        mask &= df[pattern[0]] == pattern[1]
        
    result = list(df[mask].index.values)
    
    cache[repr(r)] = result 
        
    return result
    
df.copy()

#%timeit get_cover(r, df)
#%timeit get_cover_faster(r, df)


get_cover_memoized(r, df)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 47,
 51,
 53,
 54,
 71,
 74,
 79,
 80,
 83,
 88,
 96]

In [130]:
d = {"1": 2}

d.get("1")

2

In [120]:
faster_ratio = 1.03 / 1.3

faster_ratio * 1.3

39.067 * faster_ratio

30.953084615384615

In [12]:
def get_correct_cover(r, df, Y):
    indexes_points_covered = r.get_cover(df) # indices of all points satisfying the rule
    Y_arr = pd.Series(Y)                    # make a series of all Y labels
    labels_covered_points = list(Y_arr[indexes_points_covered])   # get a list only of Y labels of the points covered
    correct_cover = []
    for ind in range(0,len(labels_covered_points)):
        if labels_covered_points[ind] == r.class_label:
            correct_cover.append(indexes_points_covered[ind])
    return correct_cover, indexes_points_covered


def get_correct_cover_faster(r, df, Y):
    indexes_points_covered = r.get_cover(df) # indices of all points satisfying the rule
    Y_arr = np.array(Y)                    # make a series of all Y labels
    labels_covered_points = Y_arr[indexes_points_covered]   # get a list only of Y labels of th
    mask = labels_covered_points == r.class_label
    result = np.array(indexes_points_covered)[mask]

    return result, indexes_points_covered


CORRECT_COVER_CACHE = {}

def get_correct_cover_memoized(r, df, Y):
    result = CORRECT_COVER_CACHE.get(repr(r))
    
    if result:
        return result
    
    indexes_points_covered = r.get_cover(df) # indices of all points satisfying the rule
    Y_arr = np.array(Y)                    # make a series of all Y labels
    labels_covered_points = Y_arr[indexes_points_covered]   # get a list only of Y labels of th
    mask = labels_covered_points == r.class_label
    result = np.array(indexes_points_covered)[mask]
    
    CORRECT_COVER_CACHE[repr(r)] = result, indexes_points_covered

    return result, indexes_points_covered
    
    
#%timeit get_correct_cover(r, df, Y)
#%timeit get_correct_cover_faster(r, df, Y)
#%timeit get_correct_cover_memoized(r, df, Y)

get_correct_cover_memoized(r, df, Y)

(array([96], dtype=int64),
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  13,
  14,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  47,
  51,
  53,
  54,
  71,
  74,
  79,
  80,
  83,
  88,
  96])

In [16]:
def overlap(r1, r2, df):
    return sorted(list(set(r1.get_cover(df)).intersection(set(r2.get_cover(df)))))


repr(r)
repr(list_of_rules[1])

'<__main__.rule object at 0x00000217CBF769E8>'

In [25]:
def get_incorrect_cover(r, df, Y):
    correct_cover, full_cover = r.get_correct_cover(df, Y)
    return (sorted(list(set(full_cover) - set(correct_cover))))


def get_incorrect_cover_faster(r, df, Y):
    correct_cover, full_cover = r.get_correct_cover(df, Y)

    mask = full_cover == correct_cover
    
    return np.array(full_cover)[~mask]


all(get_incorrect_cover_faster(r, df, Y) == get_incorrect_cover(r, df, Y))




11.6 µs ± 77.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
20.9 µs ± 581 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
